In [ ]:
# Datu ielasīšana no Influx, IP adrese ir izkļauta no koda

from influxdb import InfluxDBClient

client = InfluxDBClient(host='', port=8086)
client.switch_database('tracking')
df = pd.DataFrame(client.query('SELECT * FROM tracking').get_points())

In [ ]:
# Divu riņķu krustpunktu aprēķins. Risinājums ņemts no: 
# https://stackoverflow.com/questions/64800285/intersection-coordinates-lat-lon-of-two-circles-given-the-coordinates-of-the
from decimal import Decimal
from math import cos, sin, sqrt
import math
import numpy as np

def intersection(p1, r1_meter, p2, r2_meter):

    x_p1 = Decimal(cos(math.radians(p1[1]))*cos(math.radians(p1[0])))  # x = cos(lon)*cos(lat)
    y_p1 = Decimal(sin(math.radians(p1[1]))*cos(math.radians(p1[0])))  # y = sin(lon)*cos(lat)
    z_p1 = Decimal(sin(math.radians(p1[0])))                           # z = sin(lat)
    x1 = (x_p1, y_p1, z_p1)

    x_p2 = Decimal(cos(math.radians(p2[1]))*cos(math.radians(p2[0])))  # x = cos(lon)*cos(lat)
    y_p2 = Decimal(sin(math.radians(p2[1]))*cos(math.radians(p2[0])))  # y = sin(lon)*cos(lat)
    z_p2 = Decimal(sin(math.radians(p2[0])))                           # z = sin(lat)
    x2 = (x_p2, y_p2, z_p2)

    r1 = Decimal(math.radians((r1_meter/1852) / 60)) # r1_meter/1852 converts meter to Nautical mile.
    r2 = Decimal(math.radians((r2_meter/1852) / 60))

    q = Decimal(np.dot(x1, x2))

    if q**2 != 1 :
        a = (Decimal(cos(r1)) - Decimal(cos(r2))*q) / (1 - q**2)
        b = (Decimal(cos(r2)) - Decimal(cos(r1))*q) / (1 - q**2)

        n = np.cross(x1, x2)

        x0_1 = [a*f for f in x1]
        x0_2 = [b*f for f in x2]
        x0 = [sum(f) for f in zip(x0_1, x0_2)]

        if (np.dot(x0, x0) <= 1) & (np.dot(n,n) != 0): # This is to secure that (1 - np.dot(x0, x0)) / np.dot(n,n) > 0
            t = Decimal(sqrt((1 - np.dot(x0, x0)) / np.dot(n,n)))
            t1 = t
            t2 = -t

            i1 = x0 + t1*n
            i2 = x0 + t2*n

            i1_lat = math.degrees( math.asin(i1[2]))
            i1_lon = math.degrees( math.atan2(i1[1], i1[0] ) )
            ip1 = (i1_lat, i1_lon)

            i2_lat = math.degrees( math.asin(i2[2]))
            i2_lon = math.degrees( math.atan2(i2[1], i2[0] ) )
            ip2 = (i2_lat, i2_lon)
            return [ip1, ip2]
        elif (np.dot(n,n) == 0):
            return("The centers of the circles can be neither the same point nor antipodal points.")
        else:
            return("The circles do not intersect")
    else:
        return("The centers of the circles can be neither the same point nor antipodal points.")


In [ ]:
# Piefiksētas A un n vērtības, kā arī papildināta datu kolonna ar aprēķinātajiem attālumiem no RSSI vērtībām
A=12
n=4
df["d1"]=10**((A-df.RSSI_1)/(10*n))
df["d2"]=np.where(df.RSSI_2.isna(),0,10**((A-df.RSSI_2)/(10*n)))

In [ ]:
# Dati tika manuāli atzīmēti, lai izvilktu, kuras vērtības tika mērītas, kurās lokācijās.

kekava=df[(df.TT == 'kekava')]
centrs=df[(df.TT == 'centrs')]
sigulda=df[(df.TT == 'sigulda')]


In [ ]:
# Rezultātu vērtības (kā rezultāta koordinātas(b) un attālums no reālajām vērtībām (rez))
rez=[]
b=[]

In [ ]:
# Rezultātu iegūšana (izmantojot riņķa līnijas palielināšanau un samazināšanu, ja tie nekrustojas)

test = centrs 

for x in range(len(test)):
    small=0.95
    large=1.1
    real = (test.iloc[x]['Act_lat'], test.iloc[x]['Act_long'])
    p1 = (test.iloc[x]['Latitude_1'], test.iloc[x]['Longitude_1'])
    r1_meter = test.iloc[x]['d1']
    p2 = (test.iloc[x]['Latitude_2'], test.iloc[x]['Longitude_2'])
    r2_meter = test.iloc[x]['d2']
    a = intersection(p1, r1_meter, p2, r2_meter)
    while(a=='The circles do not intersect'):
            r1_meter=test.iloc[x]['d1']*small
            r2_meter=test.iloc[x]['d2']*small
            a = intersection(p1, r1_meter, p2, r2_meter)
            if(a=='The circles do not intersect'):
                r1_meter=test.iloc[x]['d1']*large
                r2_meter=test.iloc[x]['d2']*large
                a = intersection(p1, r1_meter, p2, r2_meter)
            small=small-0.05
            large=large+0.1
            
    b.append(a[0])
    b.append(a[1])
    rez.append(geopy.distance.geodesic(a[0],real).km)
    rez.append(geopy.distance.geodesic(a[1],real).km)


In [ ]:
# Kartē attēlotas visas aprēkinātās lokācijas
# Google API atslēga ir izkļauta no koda

import gmplot
apikey = '' 
gmap = gmplot.GoogleMapPlotter(real[0], real[1], 13, apikey=apikey)
gmap.marker(real[0], real[1], color='red') #real

for x in range(8):
    gmap.marker(b[x][0], b[x][1], face_alpha=0, ew=3, color='yellow')

gmap.draw('map.html')


In [ ]:
# Kartē attēloti aprēķinātie attālumi, kā riņķa līnijas.
# Google API atslēga ir izkļauta no koda

import gmplot
apikey = ''

gmap = gmplot.GoogleMapPlotter(test.iloc[0]['Act_lat'], test.iloc[0]['Act_long'], 13, apikey=apikey)

for x in range(len(test)):
    gmap.circle(test.iloc[x]['Latitude_1'], test.iloc[x]['Longitude_1'], test.iloc[x]['d1'], face_alpha=0, ew=3, color='red')
    gmap.circle(test.iloc[x]['Latitude_2'], test.iloc[x]['Longitude_2'], test.iloc[x]['d2'], face_alpha=0, ew=3, color='red')

gmap.marker(test.iloc[0]['Act_lat'], test.iloc[0]['Act_long'], color='red') #real
gmap.marker(test.iloc[0]['Latitude_1'], test.iloc[0]['Longitude_1'], color='green') #gtw1
gmap.marker(test.iloc[0]['Latitude_2'], test.iloc[0]['Longitude_2'], color='green') #gtw2

gmap.draw('map.html')